import necessary packages

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import os
import sys
import glob

from torch.utils.data import DataLoader
import numpy as np
import torch
from torch import optim
import matplotlib.pyplot as plt
from natsort import natsorted
from torchvision import transforms

from MIR.models import SpatialTransformer, EncoderFeatureExtractor, SITReg, VFA, TransMorphTVF, TransMorph, convex_adam_MIND
from MIR.models.SITReg import ReLUFactory, GroupNormalizerFactory
from MIR.models.SITReg.composable_mapping import DataFormat
from MIR.models.SITReg.deformation_inversion_layer.fixed_point_iteration import (
    AndersonSolver,
    AndersonSolverArguments,
    MaxElementWiseAbsStopCriterion,
    RelativeL2ErrorStopCriterion,
)
import MIR.models.convexAdam.configs_ConvexAdam_MIND as CONFIGS_CVXAdam
import MIR.models.configs_TransMorph as configs_TransMorph
import MIR.models.configs_VFA as CONFIGS_VFA
from data import datasets, trans
import torch.nn.functional as F


Define image size and dataset path

In [11]:
H, W, D = 160, 192, 224
data_dir = './IXI_data/'

Initialize models
We begin with TransMorph

In [12]:
def transmorph_model():
    scale_factor = 1
    config = configs_TransMorph.get_3DTransMorph3Lvl_config()
    config.img_size = (H//scale_factor, W//scale_factor, D//scale_factor)
    config.window_size = (H // 64, W // 64, D // 64)
    config.out_chan = 3
    print(config)
    TM_model = TransMorph(config).cuda('cuda:0')
    return TM_model

Then TransMorph-TVF

In [13]:
def transmorphTVF_model():
    scale_factor = 2
    config = configs_TransMorph.get_3DTransMorph3Lvl_config()
    config.img_size = (H//scale_factor, W//scale_factor, D//scale_factor)
    config.window_size = (H // 64, W // 64, D // 64)
    config.out_chan = 3
    print(config)
    TMTVF_model = TransMorphTVF(config, time_steps=7).cuda('cuda:0')
    return TMTVF_model

Then VFA

In [14]:
def vfa_model():
    scale_factor = 1
    config = CONFIGS_VFA.get_VFA_default_config()
    config.img_size = (H//scale_factor, W//scale_factor, D//scale_factor)
    print(config)
    VFA_model = VFA(config, device='cuda:0')
    return VFA_model

Then SITReg

In [ ]:
def create_model(INPUT_SHAPE) -> SITReg:
    """Create SITReg model from config"""
    feature_extractor = EncoderFeatureExtractor(
            n_input_channels=1,
            activation_factory=ReLUFactory(),
            n_features_per_resolution=[12, 16, 32, 64, 128, 128],
            n_convolutions_per_resolution=[2, 2, 2, 2, 2, 2],
            input_shape=INPUT_SHAPE,
            normalizer_factory=GroupNormalizerFactory(2),
        ).cuda()
    AndersonSolver_forward = AndersonSolver(
        MaxElementWiseAbsStopCriterion(min_iterations=2, max_iterations=50, threshold=1e-2),
        AndersonSolverArguments(memory_length=4),
    )
    AndersonSolver_backward = AndersonSolver(
        RelativeL2ErrorStopCriterion(min_iterations=2, max_iterations=50, threshold=1e-2),
        AndersonSolverArguments(memory_length=4),
    )
    network = SITReg(
        feature_extractor=feature_extractor,
        n_transformation_convolutions_per_resolution=[2, 2, 2, 2, 2, 2],
        n_transformation_features_per_resolution=[12, 64, 128, 256, 256, 256],
        max_control_point_multiplier=0.99,
        affine_transformation_type=None,
        input_voxel_size=(1.0, 1.0, 1.0),
        input_shape=INPUT_SHAPE,
        transformation_downsampling_factor=(1.0, 1.0, 1.0),
        forward_fixed_point_solver=AndersonSolver_forward,
        backward_fixed_point_solver=AndersonSolver_backward,
        activation_factory=ReLUFactory(),
        normalizer_factory=GroupNormalizerFactory(4),
            ).cuda()
    return network

def sitreg_model():
    return create_model((H, W, D)).cuda('cuda:0')

Then ConvexAdam-MIND

In [ ]:
def convexadam_model():
    config = CONFIGS_CVXAdam.get_ConvexAdam_MIND_brain_default_config()
    model = convex_adam_MIND
    return {'model': model, 'config': config}

Then deedsBCV (this is only available on Linux)

In [ ]:
import nibabel as nib
from MIR.utils.deedsbcv_binary import get_deedsbcv_executable

def load_deedsBCV_flow(flow_prefix: str) -> torch.Tensor:
    disp_ux = nib.load(flow_prefix + "_ux.nii.gz").get_fdata()[None, ...]
    disp_vx = nib.load(flow_prefix + "_vx.nii.gz").get_fdata()[None, ...]
    disp_wx = nib.load(flow_prefix + "_wx.nii.gz").get_fdata()[None, ...]
    disp_arr = np.concatenate([disp_vx, disp_ux, disp_wx], axis=0)
    disp_tensor = torch.from_numpy(disp_arr).float().unsqueeze(0)
    return disp_tensor

deeds_exe = get_deedsbcv_executable("deedsBCV")

Creating model library

In [ ]:
models_dict = {'TransMorph': transmorph_model,
               'TransMorphTVF': transmorphTVF_model,
               'VFA': vfa_model,
               'SITReg': sitreg_model,
               'ConvexAdam-MIND': convexadam_model}

Checking if dataset is available, if not grab from Google Drive

In [17]:
if not os.path.exists(data_dir):
    import gdown
    # download model
    file_id = '1-VQewCVNj5eTtc3eQGhTM2yXBQmgm8Ol'
    url = f"https://drive.google.com/uc?id={file_id}"
    gdown.download(url, 'IXI_data.zip', quiet=False)
    import zipfile
    with zipfile.ZipFile('IXI_data.zip', 'r') as zip:
        zip.extractall('./')
    os.remove('IXI_data.zip')

if not os.path.exists(data_dir):
    raise ValueError('Data directory not found and download failed.')

Downloading...
From (original): https://drive.google.com/uc?id=1-VQewCVNj5eTtc3eQGhTM2yXBQmgm8Ol
From (redirected): https://drive.google.com/uc?id=1-VQewCVNj5eTtc3eQGhTM2yXBQmgm8Ol&confirm=t&uuid=e766a6ce-2983-4985-accf-dbdc2bbd34d3
To: /scratch/jchen/python_projects/custom_packages/MIR/tutorials/IXI_benchmarking/IXI_data.zip
100%|██████████| 1.54G/1.54G [02:15<00:00, 11.4MB/s]


In [ ]:
test_dir = data_dir + 'test/'
atlas_dir = data_dir + 'atlas.pkl'
test_composed = transforms.Compose([
        trans.Seg_norm(),
        trans.NumpyType((np.float32, np.int16)),
    ])
test_set = datasets.IXIBrainInferDataset(glob.glob(test_dir + '*.pkl'), atlas_dir, transforms=test_composed)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=8, pin_memory=True, drop_last=True)

Initialize inference pipeline for each model

In [ ]:
spatial_trans = SpatialTransformer((H, W, D)).cuda()

def inference(model_name, model, moving, fixed):
    if model_name == "TransMorph" or model_name == 'VFA':
        with torch.no_grad():
            model.eval()
            moving = moving.cuda()
            fixed = fixed.cuda()
            flow = model((moving, fixed))
    elif model_name == "TransMorphTVF":
        with torch.no_grad():
            model.eval()
            moving = F.avg_pool3d(moving, 2).cuda()
            fixed = F.avg_pool3d(fixed, 2).cuda()
            flow = model((moving, fixed))
            flow = F.interpolate(flow, size=(H, W, D), mode='trilinear', align_corners=True) * 2.0
    elif model_name == "SITReg":
        with torch.no_grad():
            model.eval()
            moving = moving.cuda()
            fixed = fixed.cuda()
            mapping_pair = model(moving, fixed, mappings_for_levels=((0, False),))[0]
            flow = mapping_pair.forward_mapping.sample(data_format=DataFormat.voxel_displacements()).generate_values()
    elif model_name == "ConvexAdam":
        moving = moving.cuda()
        fixed = fixed.cuda()
        convexadam = model()['model']
        config = model()['config']
        flow = convexadam(moving, fixed, config)
    elif model_name == 'deedsBCV':
        os.mkdir('temp_deeds/', exist_ok=True)
        x_path = "temp_deeds/moving.nii.gz"
        y_path = "temp_deeds/fixed.nii.gz"
        out_prefix = "temp_deeds/dense_disp"

        nib.Nifti1Image(moving[0, 0].detach().cpu().numpy(), np.eye(4)).to_filename(x_path)
        nib.Nifti1Image(fixed[0, 0].detach().cpu().numpy(), np.eye(4)).to_filename(y_path)

        cmd = f"{deeds_exe} -F {y_path} -M {x_path} -O {out_prefix} -G 6x5x4x3x2 -L 6x5x4x3x2 -Q 5x4x3x2x1"
        os.system(cmd)
        flow = load_deedsBCV_flow(out_prefix).cuda()
        os.system("rm -rf temp_deeds/")
    return flow

Iterating through models

In [ ]:
from MIR.accuracy_measures import calc_J_i, calc_Jstar_1, calc_Jstar_2, calc_jac_dets, get_identity_grid, calc_measurements, dice_val_substruct

def to_cuda(batch):
    return [t.cuda(non_blocking=True) for t in batch]

for model_name, model_func in models_dict.items():
    model = model_func()
    print(f'Evaluating model: {model_name}')
    for i, sample in enumerate(test_loader):
        x, y, x_seg, y_seg = to_cuda(sample)
        x = x.float()
        y = y.float()
        flow = inference(model_name, model, x, y)
        x_seg_oh = F.one_hot(x_seg.long(), num_classes=46)
        x_seg_oh = torch.squeeze(x_seg_oh, 1)
        x_seg_oh = x_seg_oh.permute(0, 4, 1, 2, 3).contiguous()
        x_segs = []
        for i in range(46):
            def_seg = spatial_trans(x_seg_oh[:, i:i + 1, ...].float(), flow.float())
            x_segs.append(def_seg)
        x_segs = torch.cat(x_segs, dim=1)
        def_out = torch.argmax(x_segs, dim=1, keepdim=True)
        del x_segs, x_seg_oh
        
        mask = x.detach().cpu().numpy()[0, 0, 1:-1, 1:-1, 1:-1]
        mask = mask > 0
        disp_field = flow.cpu().detach().numpy()[0]
        trans_ = disp_field + get_identity_grid(disp_field)
        jac_dets = calc_jac_dets(trans_)
        non_diff_voxels, non_diff_tetrahedra, non_diff_volume = calc_measurements(jac_dets, mask)
        total_voxels = np.sum(mask)
        ndv = non_diff_volume / total_voxels * 100
        ndp = non_diff_voxels / total_voxels * 100
        
        line = dice_val_substruct(def_out.long(), y_seg.long(), stdy_idx)
        line = line +','+str(ndv)+','+str(ndp)
        #csv_writter(line, 'Quantitative_Results/' + model_folder[:-1])
    del model